In [28]:
from tracknet.model import GridTrackNetModel
from tracknet.dataset import TrackNet
from torchinfo import summary
import wandb

In [4]:
# wandb.login()

In [5]:
# wandb.init(project='GridTrackNet', name='wandb_setup_2')

model = GridTrackNetModel()

# wandb.watch(model, log_freq=500)

In [3]:
summary(model, input_size=(10, 15, 432, 768))

Layer (type:depth-idx)                   Output Shape              Param #
GridTrackNetModel                        [10, 15, 27, 48]          --
├─ConvBlock: 1-1                         [10, 64, 432, 768]        --
│    └─Sequential: 2-1                   [10, 64, 432, 768]        --
│    │    └─Conv2d: 3-1                  [10, 64, 432, 768]        8,704
│    │    └─ReLU: 3-2                    [10, 64, 432, 768]        --
│    │    └─BatchNorm2d: 3-3             [10, 64, 432, 768]        128
├─ConvBlock: 1-2                         [10, 64, 432, 768]        --
│    └─Sequential: 2-2                   [10, 64, 432, 768]        --
│    │    └─Conv2d: 3-4                  [10, 64, 432, 768]        36,928
│    │    └─ReLU: 3-5                    [10, 64, 432, 768]        --
│    │    └─BatchNorm2d: 3-6             [10, 64, 432, 768]        128
├─MaxPool2d: 1-3                         [10, 64, 216, 384]        --
├─ConvBlock: 1-4                         [10, 128, 216, 384]       --
│    └

In [6]:
import torch
import torch.nn as nn
from torch.optim import Adadelta
import numpy as np
from torch import cuda
from tqdm import tqdm

In [ ]:
def validate(model, criterion, val_loader):
    device = "cuda" if cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    corrects = []
    losses = []

    for _, (instances, label) in enumerate(val_loader):
        with torch.no_grad():
            label = label.permute(0, 1, 4, 2, 3).reshape(val_loader.batch_size, 3 * 5, 27, 48)

            instances = instances.to(device, dtype=torch.float32)
            outputs = model(instances)

            loss = criterion(outputs, label)
            losses.append(loss)

            for i in range(val_loader.batch_size):
                # Each 3 items is a grid confidence for 1 of 5 frames
                for j in range(0, 15, 3):
                    gt = np.argmax(label[i][j].flatten())
                    gt_x, gt_y = np.unravel_index(gt, (27, 48))

                    out = np.argmax(outputs[i][j].flatten())
                    out_x, out_y = np.unravel_index(out, (27, 48))

                    corrects.append(gt_x == out_x and gt_y == out_y)

    acc = sum(corrects) / len(corrects)
    avg_loss = np.average(losses)

    return acc, avg_loss

In [31]:
from torch.utils.data import DataLoader

val_dataset = TrackNet('compiled_dataset')
val_loader = DataLoader(val_dataset, batch_size=10)
validate(model, val_loader)

TypeError: TrackNet.__init__() missing 1 required positional argument: 'files'

In [ ]:

def train(model, train_loader, epochs = 10):
    device = "cuda" if cuda.is_available() else "cpu"
    model.to(device)
    model.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = Adadelta(lr=1.0, params=model.parameters(), weight_decay=0.01)

    for epoch in range(epochs):
        losses = []

        for instances, label in tqdm(train_loader):
            optimizer.zero_grad()

            label = label.permute(0, 1, 4, 2, 3).reshape(train_loader.batch_size, 3 * 5, 27, 48)

            instances = instances.to(device, dtype=torch.float32)
            outputs = model(instances)

            print(outputs.shape, label.shape)
            
            loss = criterion(outputs, label)
            losses.append(loss)

            wandb.log({ 'train_loss': loss })

            loss.backward()
            optimizer.step()
        
        avg_loss = np.mean(losses)

        wandb.log({
            'epoch': epoch,
            'epoch_loss': avg_loss
        })


In [ ]:
wandb.finish()

In [26]:
import os
import random

files = os.listdir('compiled_dataset')
files = list(filter(lambda x: x.endswith('.hdf5'), files))
random.shuffle(files)

val_files = files[:6]
train_files = files[6:]

val_files, train_files


(['015.hdf5', '035.hdf5', '041.hdf5', '038.hdf5', '046.hdf5', '008.hdf5'],
 ['036.hdf5',
  '016.hdf5',
  '051.hdf5',
  '034.hdf5',
  '030.hdf5',
  '054.hdf5',
  '009.hdf5',
  '003.hdf5',
  '047.hdf5',
  '010.hdf5',
  '001.hdf5',
  '048.hdf5',
  '022.hdf5',
  '005.hdf5',
  '027.hdf5',
  '060.hdf5',
  '025.hdf5',
  '044.hdf5',
  '032.hdf5',
  '040.hdf5',
  '006.hdf5',
  '011.hdf5',
  '055.hdf5',
  '002.hdf5',
  '021.hdf5',
  '043.hdf5',
  '056.hdf5',
  '037.hdf5',
  '004.hdf5',
  '000.hdf5',
  '053.hdf5',
  '045.hdf5',
  '028.hdf5',
  '023.hdf5',
  '007.hdf5',
  '042.hdf5',
  '057.hdf5',
  '033.hdf5',
  '059.hdf5',
  '020.hdf5',
  '018.hdf5',
  '052.hdf5',
  '017.hdf5',
  '049.hdf5',
  '039.hdf5',
  '012.hdf5',
  '061.hdf5',
  '031.hdf5',
  '058.hdf5',
  '050.hdf5',
  '024.hdf5',
  '019.hdf5',
  '029.hdf5',
  '026.hdf5',
  '014.hdf5',
  '013.hdf5'])

In [ ]:
from torch.utils.data import DataLoader

val_dataset = TrackNet('compiled_dataset', val_files, debug=True)
val_loader = DataLoader(val_dataset, batch_size=10)

train_dataset = TrackNet('compiled_dataset', train_files, debug=True)
train_loader = DataLoader(train_dataset, batch_size=10, pin_memory=True, num_workers=8)

train(model, train_loader)